In [100]:
from janome.tokenizer import Tokenizer
from collections import defaultdict
import pandas as pd
import itertools
import math

In [101]:
class NaiveBayes:
    
    # 初期化
    def __init__(self,train_file,test_file):
        # 教師データ (カテゴリ毎にファイルがある。)
        self.train_file = train_file
        self.category_num = len(self.train_file)
        
        # テストデータ
        test_file = test_file
        test_df = pd.read_csv(test_file)
        self.test_data = test_df.values.tolist()
        self.test_num = len(self.test_data)
        
        # 語彙情報
        self.text_sum = 0
        
    # 形態素解析と単語数を数える
    def _text2data(self,text_list):
        t = Tokenizer()
        vocab={}
        
        for text in text_list:
            for token in t.tokenize(text):
                word = ""
                partOfSpeech = token.part_of_speech.split(',')[0]
                if partOfSpeech == u'名詞':
                    word = token.surface
                    self.text_sum += 1
                    if word not in vocab:
                        vocab[word] = 1                
                    else:
                        vocab[word] += 1                    
        return vocab
    
    # 学習
    def train(self,train_id): 
        """
        カテゴリ内の単語出現率の計算
        - 各カテゴリーの単語数 : text_sum
        - カテゴリ内の単語出現率 : wordprob
        - カテゴリごとの単語出現率リスト : category_wordprob

        MEMO:
        ラプラススムージングスムージング追加 分子に1、分母に2
        """

        category_wordprob = {}

        #data_strat = train_id*100 
        #data_end = data_strat + 100 
        data_size = (train_id+1)*100
    
        for i,file_name in enumerate(self.train_file):

            # 教師データ用意
            train_df = pd.read_csv(file_name)
            #text_list = train_df[data_strat:data_end].values.tolist()
            text_list = train_df[:data_size].values.tolist()
            recipe_data = itertools.chain(*text_list)
            recipe_data = list(recipe_data)
            print(len(recipe_data))

            # 教師データを形態素解析
            wordcount = self._text2data(recipe_data)
    
            # カテゴリー内の単語出現率をもとめる
            wordprob = {}
            for k, v in wordcount.items():
                wordprob[k] = v+1/(self.text_sum+2)
            category_wordprob[i] = wordprob
    
        return category_wordprob

    
    # 分類
    def classifier(self,category_wordprob,p_c):

        """
        テストデータを分類する
        カテゴリ内でのテストデータの出現率の確率を求め、テストデータのカテゴリー出現率の計算する
        - カテゴリ内の文書出現率 : category_textprob    
        """
        
        success_num = 0 # 正解数
        exception = 0 # 例外数
        a = [0 for i in range(self.category_num)] # カテゴリに属するデータをカテゴリと分類した数
        b = [0 for i in range(self.category_num)] # カテゴリに属しないデータをカテゴリと分類した数
    
        next_p_o = [0 for i in range(self.test_num)] # 次の事後確率
    

        # テストデータ分の分類を行う
        for i,data in enumerate(self.test_data):
            
    
            test_text = [] # テストデータのリスト (データOの集合)
            category_textprob = [1 for i in range(self.category_num)]
            category_id = data[1]
            p_o = 0
    
            # テストデータを形態素解析
            t = Tokenizer()
            for token in t.tokenize(data[0]):
                word = ""
                partOfSpeech = token.part_of_speech.split(',')[0]
                if partOfSpeech == u'名詞':   
                    word = token.surface
                    if word not in test_text:
                        test_text.append(word)
   
            #print(str(i)+"回目",test_text,"カテゴリ："+str(category_id))
    
            # カテゴリーごとに計算
            for j in range(self.category_num): 

                # カテゴリー内のレシピ名の出現率 : p(o|c) 
                for test_word in test_text:        
                    if test_word in category_wordprob[j]:
                        category_textprob[j] *=  category_wordprob[j][test_word]               
                    else:
                        category_textprob[j] *= 1/(self.text_sum+2)                                
        
                # レシピ名の得られる確率
                p_o += category_textprob[j] * p_c[i][j]        
    
            ans_list = []
            for j in range(self.category_num):              
                #print((category_textprob[j]*p_c[i][j])/p_o)
                ans_list.append((category_textprob[j]*p_c[i][j])/p_o)        
        
            # 正解判定   
            max_ans = ans_list.index(max(ans_list))
            min_ans = ans_list.index(min(ans_list))
            if ans_list[max_ans] == ans_list[min_ans]:
                exception += 1
            elif max_ans == data[1]: # カテゴリに属するデータをカテゴリと分類した数
                success_num += 1
                a[category_id] += 1
            else: # カテゴリに属さないデータをカテゴリと分類した数
                b[max_ans] += 1

            # 次の事前確率用意
            next_p_o[i] = ans_list
        
        # 結果出力
        print("正解率："+ str(success_num/self.test_num))
        print("該当なし："+ str(exception))
        for i in range(self.category_num):
            print("----------------------------------")
            print("カテゴリid："+str(i))
            recall = a[i]/11
            print("再現率："+ str(recall))
            precision = a[i]/(a[i]+b[i])
            print("精度："+ str(precision))
            f = (2 * precision * recall)/(precision + recall)
            print("F値：" + str(f))
        
        return next_p_o


In [102]:
train_num = 9

train_file = ['./data/data_type01_main.csv','./data/data_type02_main.csv','./data/data_type03_main.csv']
test_file = './data/data_test.csv'
nb = NaiveBayes(train_file,test_file)

for train_id in range(train_num):
    print('【学習回数：{0}】'.format(train_id+1))

    p_c = [[1/3 for i in range(nb.category_num)] for j in range(nb.test_num)]
    """
    if train_id == 0:
        p_c = [[1/3 for i in range(nb.category_num)] for j in range(nb.test_num)]
    else:
        p_c = next_p_o
    """
    
    # 学習
    category_wordprob = nb.train(train_id)
    
    # 分類
    next_p_o = nb.classifier(category_wordprob,p_c)

【学習回数：1】
100
100
100
正解率：0.5454545454545454
該当なし：8
----------------------------------
カテゴリid：0
再現率：0.45454545454545453
精度：0.7142857142857143
F値：0.5555555555555556
----------------------------------
カテゴリid：1
再現率：0.7272727272727273
精度：0.6666666666666666
F値：0.6956521739130435
----------------------------------
カテゴリid：2
再現率：0.45454545454545453
精度：0.8333333333333334
F値：0.5882352941176471
【学習回数：2】
200
200
200
正解率：0.5757575757575758
該当なし：7
----------------------------------
カテゴリid：0
再現率：0.5454545454545454
精度：0.75
F値：0.631578947368421
----------------------------------
カテゴリid：1
再現率：0.5454545454545454
精度：0.6
F値：0.5714285714285713
----------------------------------
カテゴリid：2
再現率：0.6363636363636364
精度：0.875
F値：0.7368421052631579
【学習回数：3】
300
300
300
正解率：0.6363636363636364
該当なし：5
----------------------------------
カテゴリid：0
再現率：0.6363636363636364
精度：0.7
F値：0.6666666666666666
----------------------------------
カテゴリid：1
再現率：0.5454545454545454
精度：0.75
F値：0.631578947368421
------------------------------